In [1]:
import numpy as np
import pandas as pd
import datetime
import warnings

In [2]:
# options
pd.set_option('max_columns',100)
warnings.simplefilter('ignore')

In [3]:
# Load Data

#Arr_data = pd.DataFrame(pd.read_csv('C:\\Users\\user\\proj\\Terminal_Capacity\\Data\\ACMD\\IIS_Arr.dat'))
#Dep_data = pd.DataFrame(pd.read_csv('C:\\Users\\user\\proj\\Terminal_Capacity\\Data\\ACMD\\IIS_Dep.dat'))
Wx_2020 = pd.concat([pd.read_csv('..\\data\\Weather\\RKSI_air_stcs2020%02d.csv' %(i)) for i in range(1,13)], ignore_index=True)
Wx_2019 = pd.concat([pd.read_csv('..\\data\\Weather\\RKSI_air_stcs2019%02d.csv' %(i)) for i in range(1,13)], ignore_index=True)
Wx_2018 = pd.concat([pd.read_csv('..\\data\\Weather\\RKSI_air_stcs2018%02d.csv' %(i)) for i in range(1,13)], ignore_index=True)

taf_2020 = pd.read_csv('..\\data\\Weather\\TAF_data_2020.csv') 
taf_2019 = pd.read_csv('..\\data\\Weather\\TAF_data_2019.csv') 
taf_2018 = pd.read_csv('..\\data\\Weather\\TAF_data_2018.csv') 

WINTEMP_Osan_2020 = pd.DataFrame(pd.read_csv('..\\data\\WINTEMP\\UPPER_SONDE_47122_STD_2020_2020_2021.csv'))
WINTEMP_Osan_2019 = pd.DataFrame(pd.read_csv('..\\data\\WINTEMP\\UPPER_SONDE_47122_STD_2019_2019_2020.csv'))
WINTEMP_Osan_2018 = pd.DataFrame(pd.read_csv('..\\data\\WINTEMP\\UPPER_SONDE_47122_STD_2018_2018_2019.csv'))


#types = {'WDIR': int, 'WSPD': int, 'WG' : int, 'VIS' : int, 'WC' : str, 
#         'CLA_1LYR':str, 'BASE_1LYR':int, 'CLA_2LYR':str, 'BASE_2LYR':int, 'CLA_3LYR':str, 'BASE_3LYR':int}

In [4]:
Wx =  pd.concat([Wx_2018, Wx_2019, Wx_2020]).reset_index()
Wx = Wx.drop('index', axis=1)                        

In [5]:
taf =  pd.concat([taf_2018, taf_2019, taf_2020])
# taf = taf.drop('Unnamed: 0', axis=1)   

In [6]:
WINTEMP_Osan = pd.concat([WINTEMP_Osan_2018, WINTEMP_Osan_2019, WINTEMP_Osan_2020])

***
***
***

# Data Processing


## Time

In [7]:
# 24h -> 00h conversion
def twentyfour_to_zero(date):                                             
    if date[8:10] == '24':
        return pd.to_datetime(date[:-2], format = '%Y%m%d') + pd.Timedelta(days=1)   # +1d 00h 
    else:
        return pd.to_datetime(date, format = '%Y%m%d%H')


Wx.TM = Wx.TM.astype(str)                        # Wx.TM : int -> string
Wx.TM = Wx.TM.apply(twentyfour_to_zero)  

In [8]:
# Day name of the week 
DayName = Wx['TM'].dt.day_name()               
DayName = (DayName.replace('Monday', '1').replace('Tuesday', '2').replace('Wednesday', '3')
           .replace('Thursday', '4').replace('Friday', '5').replace('Saturday', '6').replace('Sunday', '7'))
DayName = pd.DataFrame({'DayName' : DayName}).reset_index().iloc[:,1]

In [9]:
# Divide time
year = Wx.TM.dt.year.reset_index().iloc[:,1]       # 항목별로 추출 -> Timestamp type X
month = Wx.TM.dt.month.reset_index().iloc[:,1]
day = Wx.TM.dt.day.reset_index().iloc[:,1]
hour = Wx.TM.dt.hour.reset_index().iloc[:,1]

In [10]:
# Time Concatenate
Time = pd.DataFrame({'year' : year, 'month' : month, 'day' : day, 'hour' : hour, 'DayName' : DayName})

***
## TAF

In [11]:
# fill NaN
taf = taf.fillna({'WDIR': 0, 'WSPD': 0, 'WG' : 0, 'VIS' : 9999, 'WC' : 0, 
            'CLA_1LYR':0, 'BASE_1LYR':400, 'CLA_2LYR':0, 'BASE_2LYR':400, 'CLA_3LYR':0, 'BASE_3LYR':400})
taf = taf.rename(columns = {'Unnamed: 0' : 'Time'})
taf['Time'] = pd.to_datetime(taf['Time'])
taf['issue_time'] = pd.to_datetime(taf['issue_time'])

In [12]:
# Same criteria
taf = taf.replace('FEW', 2)
taf = taf.replace('SCT', 4)
taf = taf.replace('BKN', 7)
taf = taf.replace('OVC', 8)

taf = taf.replace('FC', 10)   #  10 - Tornado                   
taf = taf.replace('VA', 10)
taf = taf.replace('TS', 9)    #   9 - TS                      
taf = taf.replace('SQ', 8)    #   8 - Squall                  
taf = taf.replace('PL', 7)    #   7 - Snow                      
taf = taf.replace('SN', 7)
taf = taf.replace('SG', 7)
taf = taf.replace('GR', 6)    #   6 - Freezing~, Heavy rain     
taf = taf.replace('GS', 6)
taf = taf.replace('IC', 6)
taf = taf.replace('RA', 5)    #   5 - Rain                      
taf = taf.replace('FG', 4)    #   4 - Fog                       
taf = taf.replace('DZ', 3)    #   3 - Drizzle, Haze, Duststorm  
taf = taf.replace('BR', 3)
taf = taf.replace('DS', 3)
taf = taf.replace('SS', 3)
taf = taf.replace('HZ', 3)
taf = taf.replace('SA', 3)
taf = taf.replace('DU', 3)    #   2 - others                    
taf = taf.replace('FU', 3)    #   1 - Nothing                      

taf.BASE_1LYR = taf.BASE_1LYR.replace('0', 400)
taf.BASE_2LYR = taf.BASE_2LYR.replace('0', 400)
taf.BASE_3LYR = taf.BASE_3LYR.replace('0', 400)

In [13]:
# TAF Dataframe initialize
TAF_6 = pd.DataFrame([], 
                   columns = ['WDIR_t6', 'WSPD_t6', 'WG_t6', 'VIS_t6', 'WC_t6', 'CLA_1LYR_t6', 'BASE_1LYR_t6',
                              'CLA_2LYR_t6', 'BASE_2LYR_t6','CLA_3LYR_t6', 'BASE_3LYR_t6',], index = Wx.TM)
TAF_12 = pd.DataFrame([], 
                   columns = ['WDIR_t12', 'WSPD_t12', 'WG_t12', 'VIS_t12', 'WC_t12', 'CLA_1LYR_t12', 'BASE_1LYR_t12',
                              'CLA_2LYR_t12', 'BASE_2LYR_t12','CLA_3LYR_t12', 'BASE_3LYR_t12',], index = Wx.TM)
TAF_18 = pd.DataFrame([], 
                   columns = ['WDIR_t18', 'WSPD_t18', 'WG_t18', 'VIS_t18', 'WC_t18', 'CLA_1LYR_t18', 'BASE_1LYR_t18',
                              'CLA_2LYR_t18', 'BASE_2LYR_t18','CLA_3LYR_t18', 'BASE_3LYR_t18',], index = Wx.TM)
TAF_24 = pd.DataFrame([], 
                   columns = ['WDIR_t24', 'WSPD_t24', 'WG_t24', 'VIS_t24', 'WC_t24', 'CLA_1LYR_t24', 'BASE_1LYR_t24',
                              'CLA_2LYR_t24', 'BASE_2LYR_t24','CLA_3LYR_t24', 'BASE_3LYR_t24',], index = Wx.TM)

In [14]:
taf_sort = taf.sort_values('Time')

for i in range(len(Wx)):
    temp_date = datetime.datetime(2018,1,1,0,0) + datetime.timedelta(hours = i)   
    temp_taf_date = taf_sort[taf_sort['Time'] == temp_date].reset_index()  
    total_taf_eachdate = temp_taf_date.drop('index',1).drop('Time',1).sort_values('issue_time').reset_index().drop('index',1)

    # 6시간 전 예보
    taf_6 = total_taf_eachdate[(total_taf_eachdate['issue_time'] < temp_date) 
                               & (total_taf_eachdate['issue_time'] >= temp_date - datetime.timedelta(hours=6))][0:1].drop('issue_time',1)
    if taf_6.empty:
        TAF_6[temp_date:temp_date] = TAF_6[temp_date:temp_date]
    else:
        TAF_6[temp_date:temp_date] = taf_6

    # 12시간 전 예보
    taf_12 = total_taf_eachdate[(total_taf_eachdate['issue_time'] < temp_date - datetime.timedelta(hours=6)) 
                                & (total_taf_eachdate['issue_time'] >= temp_date - datetime.timedelta(hours=12))][0:1].drop('issue_time',1)
    if taf_12.empty:
        TAF_12[temp_date:temp_date] = TAF_12[temp_date:temp_date]
    else:
        TAF_12[temp_date:temp_date] = taf_12

    # 18시간 전 예보
    taf_18 = total_taf_eachdate[(total_taf_eachdate['issue_time'] < temp_date - datetime.timedelta(hours=12)) 
                                & (total_taf_eachdate['issue_time'] >= temp_date - datetime.timedelta(hours=18))][0:1].drop('issue_time',1)
    if taf_18.empty:
        TAF_18[temp_date:temp_date] = TAF_18[temp_date:temp_date]
    else:
        TAF_18[temp_date:temp_date] = taf_18

    # 24시간 전 예보
    taf_24 = total_taf_eachdate[(total_taf_eachdate['issue_time'] < temp_date - datetime.timedelta(hours=18)) 
                                & (total_taf_eachdate['issue_time'] >= temp_date - datetime.timedelta(hours=24))][0:1].drop('issue_time',1)
    if taf_24.empty:
        TAF_24[temp_date:temp_date] = TAF_24[temp_date:temp_date]
    else:
        TAF_24[temp_date:temp_date] = taf_24
        
        
TAF_6 = TAF_6.fillna({'WDIR_t6': 0, 'WSPD_t6': 0, 'WG_t6' : 0, 'VIS_t6' : 9999, 'WC_t6' : 0, 
                      'CLA_1LYR_t6':0, 'BASE_1LYR_t6':400, 'CLA_2LYR_t6':0, 'BASE_2LYR_t6':400,
                      'CLA_3LYR_t6':0, 'BASE_3LYR_t6':400})
TAF_12 = TAF_12.fillna({'WDIR_t12': 0, 'WSPD_t12': 0, 'WG_t12' : 0, 'VIS_t12' : 9999, 'WC_t12' : 0, 
                        'CLA_1LYR_t12':0, 'BASE_1LYR_t12':400, 'CLA_2LYR_t12':0, 'BASE_2LYR_t12':400,
                        'CLA_3LYR_t12':0, 'BASE_3LYR_t12':400})
TAF_18 = TAF_18.fillna({'WDIR_t18': 0, 'WSPD_t18': 0, 'WG_t18' : 0, 'VIS_t18' : 9999, 'WC_t18' : 0, 
                        'CLA_1LYR_t18':0, 'BASE_1LYR_t18':400, 'CLA_2LYR_t18':0, 'BASE_2LYR_t18':400,
                        'CLA_3LYR_t18':0, 'BASE_3LYR_t18':400})
TAF_24 = TAF_24.fillna({'WDIR_t24': 0, 'WSPD_t24': 0, 'WG_t24' : 0, 'VIS_t24' : 9999, 'WC_t24' : 0, 
                        'CLA_1LYR_t24':0, 'BASE_1LYR_t24':400, 'CLA_2LYR_t24':0, 'BASE_2LYR_t24':400,
                        'CLA_3LYR_t24':0, 'BASE_3LYR_t24':400})
    
TAF_6 = TAF_6.reset_index().drop('TM',1)
TAF_12 = TAF_12.reset_index().drop('TM',1)
TAF_18 = TAF_18.reset_index().drop('TM',1)
TAF_24 = TAF_24.reset_index().drop('TM',1)

***
## METAR

In [15]:
# Present Weather code

# <Wx> Present weather (code 4677)
#
#  00-49 : No Precipitation at the station at the time of the observation
#  00-19 : No precipitation, fog, ice fog (except for 11 and 12), duststorm, 
#          sandstorm, drifting or blowing snow at the station* at the time of 
#          observation or, except for 09 and 17, during the preceding hour
#  20-29 : Precipitation, fog, ice fog or thunderstorm at the station during
#          the preceding hour but not at the time of observation
#  30-39 : Duststorm, sandstorm, drifting or blowing snow
#  40-49 : Fog or ice fog at the time of observation
#  50-59 : Drizzle
#  60-69 : Rain
#  70-79 : Solid precipitation not in showers
#  80-99 : Showery precipitation, or precipitation with current 
#          or recent thunderstorm

# Order of severity (Code 4677)
#  10 - Tornado                   (19)
#   9 - TS                        (13, 17, 91-99)
#   8 - Squall                    (18)
#   7 - Snow                      (70-78, 83-90) 
#   6 - Freezing~, Heavy rain     (56-57, 66-69, 79)
#   5 - Rain                      (58-59, 60-65, 80-82) 
#   4 - Fog                       (40-49)
#   3 - Drizzle, Haze, Duststorm  (5, 10, 30-39, 50-55)
#   2 - others                    (0-4, 6-9, 11-12, 14-16)
#   1 - Nothing                   (NaN)
#                                 (20번대는 제외)

def present_weather(weather):
    if weather == 19:
        weather = 10
    elif weather == 13 or weather == 17 or weather in range(91,100):
        weather = 9
    elif weather == 18:
        weather = 8
    elif weather in range(70, 79) or weather in range(83,91):
        weather = 7
    elif weather in range(56, 58) or weather in range(66,70) or weather == 79:
        weather = 6
    elif weather in range(58, 60) or weather in range(60,66) or weather in range(80-83):
        weather = 5
    elif weather in range(40, 50):
        weather = 4
    elif weather == 5 or weather == 10 or weather in range(30, 40) or weather in range(50,56):
        weather = 3
    elif weather in range(0,5) or weather in range(6,10) or weather in range(11, 13) or weather in range(14,17):
        weather = 2        
    else:
        weather = 1
    return weather

Wx['WC'] = Wx['WC'].apply(present_weather)

In [16]:
# Fill NA/NaN
Wx = Wx.fillna({'WS_GST' : 0, 'RVR1' : 1000, 'RVR2' : 1000, 'RVR3' : 1000, 'RN' : 0,
                'BASE_1LYR' : 400, 'BASE_2LYR' : 400, 'BASE_3LYR' : 400,'BASE_4LYR' : 400,
                'CLA_1LYR' : 0,'CLA_2LYR' : 0,'CLA_3LYR' : 0,'CLA_4LYR' : 0})

In [17]:
# Mean RVR
RVR = Wx.loc[:,'RVR1':'RVR3'].min(axis=1)
RVR = pd.DataFrame({'RVR':RVR})
Wx = Wx.join(RVR)

In [18]:
# Remove useless column
Wx = Wx.drop('CLF_1LYR', axis=1)                        
Wx = Wx.drop('CLF_2LYR', axis=1) 
Wx = Wx.drop('CLF_3LYR', axis=1)                        
Wx = Wx.drop('CLF_4LYR', axis=1)
Wx = Wx.drop('RVR1', axis=1)                        
Wx = Wx.drop('RVR2', axis=1)                        
Wx = Wx.drop('RVR3', axis=1)
Wx = Wx.drop('RVR4', axis=1)                        
Wx = Wx.drop('TM', axis=1)                        # datetime 버리기
METAR = Wx

***
## Airport Condition / Previous Airpot Condition

In [19]:
# Ceiling
CIG = np.zeros((len(Wx),1))
for i in range(len(Wx)):
    if Wx.CLA_1LYR[i] >= 5:
        CIG[i] = Wx.BASE_1LYR[i]
    elif Wx.CLA_2LYR[i] >= 5:
        CIG[i] = Wx.BASE_2LYR[i]
    elif Wx.CLA_3LYR[i] >= 5:
        CIG[i] = Wx.BASE_3LYR[i]
    elif Wx.CLA_4LYR[i] >= 5:
        CIG[i] = Wx.BASE_4LYR[i]
    else: 
        CIG[i] = 400

In [20]:
# VFR or IFR
P_Airp = np.zeros((len(Wx),1))            # VFR = 1 / MVFR = 2 / IFR = 3 / LIFR = 4
for i in range(len(Wx)):
    if Wx.VIS[i] > 600 and CIG[i] > 30:
        P_Airp[i] = 1
    elif Wx.VIS[i] >= 480 and Wx.VIS[i] <= 600 or CIG[i] >= 10 and CIG[i] <= 30:
        P_Airp[i] = 2
    elif Wx.VIS[i] >= 160 and Wx.VIS[i] < 480 or CIG[i] >= 5 and CIG[i] < 30:
        P_Airp[i] = 3
    else :
        P_Airp[i] = 4
#    elif Wx.VIS[i] < 160 and CIG[i] < 5:
#        P_Airp[i] = 4   

In [21]:
Arpt_cond = P_Airp.flatten()    # Arpt_cond = P_Airp.reshape(8760,)    flatten() : 2차원배열을 1차원으로 만들어 주는 거 (100,1) -> (100,)
Arpt_cond = pd.DataFrame({'Arpt_cond' : Arpt_cond})
P_Airp = P_Airp.flatten()     # P_Airp = P_Airp.reshape(8760,)
P_Airp = np.roll(P_Airp, 1)                  # 맨 처음 data에는 0 넣음
P_Airp[0] = 1
P_Airp = pd.DataFrame({'P_Airp' : P_Airp})

***
## Actual Arrival Rate (AAR) / Estimate Arrival Demand (EAD)

In [22]:
EAD = np.zeros(365*24*2)  
AAR = np.zeros(365*24*2)

start = '2018-01-01'
end = '2019-12-31'
j = 0

datelist = pd.date_range(start, end).astype(str).to_list()

for date in datelist:
    
    Arr_data = pd.read_csv(f'..\\data\\FOIS\\Arrival_{date}.csv', index_col= 0)
    Arr_data['Arrival_SCH'] = date + Arr_data['Arrival_SCH']        # 해당 날짜 더하기
    Arr_data['Arrival_SCH'] = pd.to_datetime(Arr_data['Arrival_SCH'], format='%Y-%m-%d%H:%M')        # datetime으로 변환
    Arr_data['Arrival_ATA'] = date + Arr_data['Arrival_ATA']        # 해당 날짜 더하기
    Arr_data['Arrival_ATA'] = pd.to_datetime(Arr_data['Arrival_ATA'], format='%Y-%m-%d%H:%M')        # datetime으로 변환
    for i in range(24):
        EAD[j] = len(Arr_data[(Arr_data['Arrival_SCH'] >= datetime.datetime.strptime(date, '%Y-%m-%d') + datetime.timedelta(hours = i))
                               & (Arr_data['Arrival_SCH'] < datetime.datetime.strptime(date, '%Y-%m-%d') + datetime.timedelta(hours = i+1))])
        AAR[j] = len(Arr_data[(Arr_data['Arrival_ATA'] >= datetime.datetime.strptime(date, '%Y-%m-%d') + datetime.timedelta(hours = i))
                               & (Arr_data['Arrival_ATA'] < datetime.datetime.strptime(date, '%Y-%m-%d') + datetime.timedelta(hours = i+1))])
        j = j+1  

In [23]:
# 2020년 - 윤년
arr_data = pd.read_excel('../data/FOIS/RKSI_19_20_arrival.xlsx')
arr_data = arr_data[arr_data['STA_DATE'] > 20200100].reset_index(drop=True)
nan_index = arr_data[arr_data['ATA'] == False].index
arr_data['ATA'] = arr_data['ATA'].fillna(0)

arr_data['Arrival_SCH'] = pd.to_datetime(arr_data['STA_DATE'].astype(str) + arr_data['STA'].astype(str).str.zfill(4), format = '%Y%m%d%H%M')
arr_data['Arrival_ATA'] = pd.to_datetime(arr_data['STA_DATE'].astype(str) + arr_data['ATA'].astype(int).astype(str).str.zfill(4), format = '%Y%m%d%H%M')
arr_data['Arrival_ATA'][nan_index] = np.nan

EAD_2020 = np.zeros(366*24)  
AAR_2020 = np.zeros(366*24)

start = '20200101'
end = '20201231'

k = 0

datelist = pd.date_range(start, end).strftime('%Y%m%d').astype(str).to_list()

for date in datelist:
    temp_arr_data = arr_data[arr_data['STA_DATE'] == int(date)]
    
    for i in range(24):
        EAD_2020[k] = len(temp_arr_data[(temp_arr_data['Arrival_SCH'] >= datetime.datetime.strptime(date, '%Y%m%d') + datetime.timedelta(hours = i))
                                        & (temp_arr_data['Arrival_SCH'] < datetime.datetime.strptime(date, '%Y%m%d') + datetime.timedelta(hours = i+1))])
        AAR_2020[k] = len(temp_arr_data[(temp_arr_data['Arrival_ATA'] >= datetime.datetime.strptime(date, '%Y%m%d') + datetime.timedelta(hours = i))
                                        & (temp_arr_data['Arrival_ATA'] < datetime.datetime.strptime(date, '%Y%m%d') + datetime.timedelta(hours = i+1))])
        k = k+1

In [24]:
EAD = np.append(EAD, EAD_2020)
AAR = np.append(AAR, AAR_2020)

In [25]:
"""
# ACMD data

Arr_data = Arr_data[['SDT','ELDT','ALDT','RWY']]                           # ELDT : 예상착륙시간,  ALDT : 실제착륙시간
Arr_data.ELDT = Arr_data.SDT + Arr_data.ELDT                               # SDT에는 연월일, ELDT/ALDT에는 시분이 나오므로 합친 뒤 
Arr_data.ELDT = pd.to_datetime(Arr_data.ELDT, format = '%Y-%m-%d%H:%M')    # datetime으로 변환
Arr_data.ALDT = Arr_data.SDT + Arr_data.ALDT                               
Arr_data.ALDT = pd.to_datetime(Arr_data.ALDT, format = '%Y-%m-%d%H:%M')   


EAD2 = np.zeros(len(Wx))  # 위에꺼랑 비교하려고
AAR2 = np.zeros(len(Wx))

for i in range(len(Wx)):
    EAD2[i] = len(Arr_data[(Arr_data.ELDT >= datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i))
                          & (Arr_data.ELDT < datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i+1))])
    AAR2[i] = len(Arr_data[(Arr_data.ALDT >= datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i))
                          & (Arr_data.ALDT < datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i+1))])
                           # 결항인 것은 AAR값이 없음 -> (EAD - AAR)에서 NaN은 결항

"""

"\n# ACMD data\n\nArr_data = Arr_data[['SDT','ELDT','ALDT','RWY']]                           # ELDT : 예상착륙시간,  ALDT : 실제착륙시간\nArr_data.ELDT = Arr_data.SDT + Arr_data.ELDT                               # SDT에는 연월일, ELDT/ALDT에는 시분이 나오므로 합친 뒤 \nArr_data.ELDT = pd.to_datetime(Arr_data.ELDT, format = '%Y-%m-%d%H:%M')    # datetime으로 변환\nArr_data.ALDT = Arr_data.SDT + Arr_data.ALDT                               \nArr_data.ALDT = pd.to_datetime(Arr_data.ALDT, format = '%Y-%m-%d%H:%M')   \n\n\nEAD2 = np.zeros(len(Wx))  # 위에꺼랑 비교하려고\nAAR2 = np.zeros(len(Wx))\n\nfor i in range(len(Wx)):\n    EAD2[i] = len(Arr_data[(Arr_data.ELDT >= datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i))\n                          & (Arr_data.ELDT < datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i+1))])\n    AAR2[i] = len(Arr_data[(Arr_data.ALDT >= datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i))\n                          & (Arr_data.ALDT < datetime.datetime(2019,1,1,0,0) +

***
## Actual Departure Rate (ADR) / Estimate Departure Demand (EDD)

In [26]:
EDD = np.zeros(365*24*2)  
ADR = np.zeros(365*24*2)

start = '2018-01-01'
end = '2019-12-31'
j = 0

datelist = pd.date_range(start, end).astype(str).to_list()

for date in datelist:
    
    dep_data = pd.read_csv(f'..\\data\\FOIS\\Departure_{date}.csv', index_col= 0)
    dep_data['Departure_SCH'] = date + dep_data['Departure_SCH']        # 해당 날짜 더하기
    dep_data['Departure_SCH'] = pd.to_datetime(dep_data['Departure_SCH'], format='%Y-%m-%d%H:%M')        # datetime으로 변환
    dep_data['Departure_ATD'] = date + dep_data['Departure_ATD']        # 해당 날짜 더하기
    dep_data['Departure_ATD'] = pd.to_datetime(dep_data['Departure_ATD'], format='%Y-%m-%d%H:%M')        # datetime으로 변환
    for i in range(24):
        EDD[j] = len(dep_data[(dep_data['Departure_SCH'] >= datetime.datetime.strptime(date, '%Y-%m-%d') + datetime.timedelta(hours = i))
                               & (dep_data['Departure_SCH'] < datetime.datetime.strptime(date, '%Y-%m-%d') + datetime.timedelta(hours = i+1))])
        ADR[j] = len(dep_data[(dep_data['Departure_ATD'] >= datetime.datetime.strptime(date, '%Y-%m-%d') + datetime.timedelta(hours = i))
                               & (dep_data['Departure_ATD'] < datetime.datetime.strptime(date, '%Y-%m-%d') + datetime.timedelta(hours = i+1))])
        j = j+1

In [27]:
# 2020년 - 윤년
dep_data = pd.read_excel('../data/FOIS/RKSI_19_20_departure.xlsx')
dep_data = dep_data[dep_data['SCH_DATE'] > 20200100].reset_index(drop=True)
nan_index = dep_data[dep_data['ATD'] == False].index
dep_data['ATD'] = dep_data['ATD'].fillna(0)

dep_data['Departure_SCH'] = pd.to_datetime(dep_data['SCH_DATE'].astype(str) + dep_data['SCH_TIME'].astype(str).str.zfill(4), format = '%Y%m%d%H%M')
dep_data['Departure_ATA'] = pd.to_datetime(dep_data['SCH_DATE'].astype(str) + dep_data['ATD'].astype(int).astype(str).str.zfill(4), format = '%Y%m%d%H%M')
dep_data['Departure_ATA'][nan_index] = np.nan

EDD_2020 = np.zeros(366*24)  
ADR_2020 = np.zeros(366*24)

start = '20200101'
end = '20201231'

k = 0

datelist = pd.date_range(start, end).strftime('%Y%m%d').astype(str).to_list()

for date in datelist:
    temp_dep_data = dep_data[dep_data['SCH_DATE'] == int(date)]
    
    for i in range(24):
        EDD_2020[k] = len(temp_dep_data[(temp_dep_data['Departure_SCH'] >= datetime.datetime.strptime(date, '%Y%m%d') + datetime.timedelta(hours = i))
                                        & (temp_dep_data['Departure_SCH'] < datetime.datetime.strptime(date, '%Y%m%d') + datetime.timedelta(hours = i+1))])
        ADR_2020[k] = len(temp_dep_data[(temp_dep_data['Departure_ATA'] >= datetime.datetime.strptime(date, '%Y%m%d') + datetime.timedelta(hours = i))
                                        & (temp_dep_data['Departure_ATA'] < datetime.datetime.strptime(date, '%Y%m%d') + datetime.timedelta(hours = i+1))])
        k = k+1

In [28]:
EDD = np.append(EDD, EDD_2020)
ADR = np.append(ADR, ADR_2020)

In [29]:
"""
# EOBT / AOBT : Pushback Time
# EROT / AROT : Ramp Out Time
# EXOT / AXOT : Taxi Out Time (Ramp to RWY time)
# ETOT / ATOT : Takeoff Time (ETOT = EOBT or VTT + EXOT)       
# (* EPIT / APIT, EPOT / APOT : Deicing in, out time )


# EXOT, AXOT 사용해서 Ground Control의 혼잡도 표현 가능??  ->  AXOT - EXOT 로 표현하면 되나? 
# ETOT 가 flight plan의 scheduled time이 아닌가??         ->  



Arr_data = Arr_data[['SDT','ELDT','ALDT','RWY']]                           # ELDT : 예상착륙시간,  ALDT : 실제착륙시간
Arr_data.ELDT = Arr_data.SDT + Arr_data.ELDT                               # SDT에는 연월일, ELDT/ALDT에는 시분이 나오므로 합친 뒤 
Arr_data.ELDT = pd.to_datetime(Arr_data.ELDT, format = '%Y-%m-%d%H:%M')    # datetime으로 변환
Arr_data.ALDT = Arr_data.SDT + Arr_data.ALDT                               
Arr_data.ALDT = pd.to_datetime(Arr_data.ALDT, format = '%Y-%m-%d%H:%M')   


EAD = np.zeros(len(Wx))
AAR = np.zeros(len(Wx))

for i in range(len(Wx)):
    EAD[i] = len(Arr_data[(Arr_data.ELDT >= datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i))
                          & (Arr_data.ELDT < datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i+1))])
    AAR[i] = len(Arr_data[(Arr_data.ALDT >= datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i))
                          & (Arr_data.ALDT < datetime.datetime(2019,1,1,0,0) + datetime.timedelta(hours = i+1))])
                           # 결항인 것은 AAR값이 없음 -> (EAD - AAR)에서 NaN은 결항
                           
"""

"\n# EOBT / AOBT : Pushback Time\n# EROT / AROT : Ramp Out Time\n# EXOT / AXOT : Taxi Out Time (Ramp to RWY time)\n# ETOT / ATOT : Takeoff Time (ETOT = EOBT or VTT + EXOT)       \n# (* EPIT / APIT, EPOT / APOT : Deicing in, out time )\n\n\n# EXOT, AXOT 사용해서 Ground Control의 혼잡도 표현 가능??  ->  AXOT - EXOT 로 표현하면 되나? \n# ETOT 가 flight plan의 scheduled time이 아닌가??         ->  \n\n\n\nArr_data = Arr_data[['SDT','ELDT','ALDT','RWY']]                           # ELDT : 예상착륙시간,  ALDT : 실제착륙시간\nArr_data.ELDT = Arr_data.SDT + Arr_data.ELDT                               # SDT에는 연월일, ELDT/ALDT에는 시분이 나오므로 합친 뒤 \nArr_data.ELDT = pd.to_datetime(Arr_data.ELDT, format = '%Y-%m-%d%H:%M')    # datetime으로 변환\nArr_data.ALDT = Arr_data.SDT + Arr_data.ALDT                               \nArr_data.ALDT = pd.to_datetime(Arr_data.ALDT, format = '%Y-%m-%d%H:%M')   \n\n\nEAD = np.zeros(len(Wx))\nAAR = np.zeros(len(Wx))\n\nfor i in range(len(Wx)):\n    EAD[i] = len(Arr_data[(Arr_data.ELDT >= datetime.datetime(2019,1,

***
## Previous hour's AAR / ADR

In [30]:
# Previous AAR
P_AAR = np.roll(AAR,1)                  # 맨 처음 data에는 0 넣음
P_AAR[0] = 0
P_AAR = pd.DataFrame({'P_AAR' : P_AAR})

# Previous ADR
P_ADR = np.roll(ADR,1)
P_ADR[0] = 0
P_ADR = pd.DataFrame({'P_ADR' : P_ADR})

***
## Wind/Temp

In [31]:
WINTEMP_Osan['Time'] = pd.to_datetime(WINTEMP_Osan['Time'], format='%d/%m/%Y %H:%M', infer_datetime_format=True)
WINTEMP_Osan_app_alt = WINTEMP_Osan[WINTEMP_Osan['Pressure'] > 300]
WINTEMP_Osan_app_alt = WINTEMP_Osan_app_alt.fillna(0)

WINTEMP = pd.DataFrame()

for i in range(0,6765,6):
    wintemp = WINTEMP_Osan_app_alt[i:i+6]
    data = pd.DataFrame(pd.concat([wintemp['WD'],wintemp['WS']], axis = 0)).reset_index().drop('index', axis=1).T
    WINTEMP = WINTEMP.append(pd.concat([data]*6))

for i in range(6765,len(WINTEMP_Osan_app_alt),6):   # 10월 10일 부터 2번씩 관측함
    wintemp = WINTEMP_Osan_app_alt[i:i+6]
    data = pd.DataFrame(pd.concat([wintemp['WD'],wintemp['WS']], axis = 0)).reset_index().drop('index', axis=1).T
    WINTEMP = WINTEMP.append(pd.concat([data]*12))
WINTEMP = WINTEMP.reset_index(drop=True)
WINTEMP = WINTEMP.rename(columns ={0:'WD_400',1:'WD_500',2:'WD_700',3:'WD_850',4:'WD_925',5:'WD_1000',
                                   6:'WS_400',7:'WS_500',8:'WS_700',9:'WS_850',10:'WS_925',11:'WS_1000' } )
WINTEMP = WINTEMP.fillna(0)

***

## Remainder

In [32]:
# remainder = previos hour's demand - previous hour's actual AAR 
arrival_remainder = EAD - AAR
arrival_remainder = np.roll(arrival_remainder,1)  
arrival_remainder[0] = 0
arrival_remainder = pd.DataFrame({'Arrival_remainder' : arrival_remainder})
departure_remainder = EDD - ADR
departure_remainder = np.roll(departure_remainder,1)  
departure_remainder[0] = 0
departure_remainder = pd.DataFrame({'Departure_remainder' : departure_remainder})

In [33]:
# 캔슬된 수 구하기

#fois_arrival = pd.read_excel('..\\data\\FOIS\\RKSI_19_20_arrival.xlsx')
#fois_departure = pd.read_excel('..\\data\\FOIS\\RKSI_19_20_departure.xlsx')

fois_arrival_2019 = pd.read_excel('..\\data\\FOIS\\RKSI_19_20_arrival.xlsx')        #2020도 포함됨
fois_departure_2019 = pd.read_excel('..\\data\\FOIS\\RKSI_19_20_departure.xlsx')
fois_arrival_2018 = pd.read_excel('..\\data\\FOIS\\RKSI_18_arrival.xlsx')
fois_departure_2018 = pd.read_excel('..\\data\\FOIS\\RKSI_18_departure.xlsx')

fois_arrival = pd.concat([fois_arrival_2018, fois_arrival_2019]).reset_index(drop=True)
fois_departure = pd.concat([fois_departure_2018, fois_departure_2019]).reset_index(drop=True)


fois_arrival['Time'] = pd.to_datetime(fois_arrival['STA_DATE'].astype(str) + fois_arrival['STA'].astype(str).str.zfill(4), 
                                      format = '%Y%m%d%H%M') #STA를 4자리로 만들고 합친 뒤, datetime으로 바꿈
fois_arrival = fois_arrival.drop('STA_DATE', axis=1)
fois_arrival = fois_arrival.drop('STA', axis=1)

fois_departure['Time'] = pd.to_datetime(fois_departure['SCH_DATE'].astype(str) + fois_departure['SCH_TIME'].astype(str).str.zfill(4), 
                                        format = '%Y%m%d%H%M') #STA를 4자리로 만들고 합친 뒤, datetime으로 바꿈
fois_departure = fois_departure.drop('SCH_DATE', axis=1)
fois_departure = fois_departure.drop('SCH_TIME', axis=1)

# fois_departure['DEP_STATUS'].value_counts()   # DEP, DLA, CNL, DIV
# fois_arrival['ARR_STATUS'].value_counts()     # DEP, DLA, CNL, DIV, LND

In [34]:
cnl_a = np.zeros(len(Wx))
cnl_d = np.zeros(len(Wx))

for i in range(len(Wx)):
    time = datetime.datetime(2018,1,1) + datetime.timedelta(hours = i+1)
    # 해당되는 시간으로 나눔
    temp_a = fois_arrival[(fois_arrival['Time'] < time) & (fois_arrival['Time'] >= time-datetime.timedelta(hours=1))] 
    temp_d = fois_departure[(fois_departure['Time'] < time) & (fois_departure['Time'] >= time-datetime.timedelta(hours=1))]
    # 해당 시간에서 캔슬 수 카운트
    cnl_a[i] = sum((temp_a['ARR_STATUS'] == 'CNL')|(temp_a['ARR_STATUS'] == 'DIV'))      
    cnl_d[i] = sum((temp_d['DEP_STATUS'] == 'CNL'))

In [35]:
# 캔슬 수 빼기
arrival_remainder['Arrival_remainder'] = arrival_remainder['Arrival_remainder'] - cnl_a
departure_remainder['Departure_remainder'] = departure_remainder['Departure_remainder'] - cnl_d

In [36]:
# 0보다 작은 수 나온거면 0으로 바꿈
arrival_remainder[arrival_remainder < 0] = 0
departure_remainder[departure_remainder < 0] = 0

***
***
***


# Data Concatenate


In [37]:
# Data Concatenate
Data = pd.DataFrame({'AAR':AAR, 'EAD':EAD, 'ADR':ADR, 'EDD':EDD})
Data = Data.join(Time)
Data = Data.join(Arpt_cond)
Data = Data.join(P_Airp)
Data = Data.join(P_AAR)
Data = Data.join(P_ADR)
Data = Data.join(arrival_remainder)
Data = Data.join(departure_remainder)
Data = Data.join(WINTEMP)
Data = Data.join(METAR)
Data = Data.join(TAF_6)
Data = Data.join(TAF_12)
Data = Data.join(TAF_18)
Data = Data.join(TAF_24)

In [40]:
pd.options.display.max_columns = None
Data[20000:20010]

,AAR,EAD,ADR,EDD,year,month,day,hour,DayName,Arpt_cond,P_Airp,P_AAR,P_ADR,Arrival_remainder,Departure_remainder,WD_400,WD_500,WD_700,WD_850,WD_925,WD_1000,WS_400,WS_500,WS_700,WS_850,WS_925,WS_1000,WD,WSPD,WS_GST,VIS,WC,TMP,TD,PS,PA,RN,HM,CA_TOT,CLA_1LYR,BASE_1LYR,CLA_2LYR,BASE_2LYR,CLA_3LYR,BASE_3LYR,CLA_4LYR,BASE_4LYR,RVR,WDIR_t6,WSPD_t6,WG_t6,VIS_t6,WC_t6,CLA_1LYR_t6,BASE_1LYR_t6,CLA_2LYR_t6,BASE_2LYR_t6,CLA_3LYR_t6,BASE_3LYR_t6,WDIR_t12,WSPD_t12,WG_t12,VIS_t12,WC_t12,CLA_1LYR_t12,BASE_1LYR_t12,CLA_2LYR_t12,BASE_2LYR_t12,CLA_3LYR_t12,BASE_3LYR_t12,WDIR_t18,WSPD_t18,WG_t18,VIS_t18,WC_t18,CLA_1LYR_t18,BASE_1LYR_t18,CLA_2LYR_t18,BASE_2LYR_t18,CLA_3LYR_t18,BASE_3LYR_t18,WDIR_t24,WSPD_t24,WG_t24,VIS_t24,WC_t24,CLA_1LYR_t24,BASE_1LYR_t24,CLA_2LYR_t24,BASE_2LYR_t24,CLA_3LYR_t24,BASE_3LYR_t24
20000,2.0,6.0,1.0,1.0,2020,4,13,8,1,1.0,1.0,7.0,4.0,1.0,0.0,170.0,190.0,140.0,95.0,105.0,110.0,23.0,8.0,13.0,9.0,20.0,21.0,2,8,0.0,1000,1,74,-8,10155,10147,0.0,56.0,5,5.0,180.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,10.0,10.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,10.0,10.0,0.0,9999.0,0,2,30.0,4,150.0,0,400.0,10.0,10.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0,30.0,15.0,25.0,6000.0,0,7,35.0,0,400.0,0,400.0
20001,2.0,2.0,5.0,5.0,2020,4,13,9,1,1.0,1.0,2.0,1.0,4.0,0.0,170.0,190.0,140.0,95.0,105.0,110.0,23.0,8.0,13.0,9.0,20.0,21.0,36,5,0.0,1000,1,90,-13,10158,10150,0.0,48.0,5,5.0,180.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,10.0,10.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,10.0,10.0,0.0,9999.0,0,2,30.0,4,150.0,0,400.0,10.0,10.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0,30.0,15.0,25.0,6000.0,0,7,35.0,0,400.0,0,400.0
20002,3.0,2.0,4.0,3.0,2020,4,13,10,1,1.0,1.0,2.0,5.0,0.0,0.0,170.0,190.0,140.0,95.0,105.0,110.0,23.0,8.0,13.0,9.0,20.0,21.0,33,7,0.0,1000,1,107,-14,10159,10151,0.0,43.0,5,5.0,180.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,360.0,7.0,0.0,9999.0,0,2,30.0,4,150.0,0,400.0,10.0,10.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,10.0,10.0,0.0,9999.0,0,2,30.0,4,150.0,0,400.0,10.0,10.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0
20003,3.0,5.0,0.0,1.0,2020,4,13,11,1,1.0,1.0,3.0,4.0,0.0,0.0,170.0,190.0,140.0,95.0,105.0,110.0,23.0,8.0,13.0,9.0,20.0,21.0,32,8,0.0,1000,1,125,-20,10157,10149,0.0,36.0,3,3.0,180.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,300.0,10.0,0.0,9999.0,0,0,0.0,0,0.0,0,0.0,310.0,8.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,10.0,10.0,0.0,9999.0,0,2,30.0,4,150.0,0,400.0,10.0,10.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0
20004,5.0,4.0,3.0,6.0,2020,4,13,12,1,1.0,1.0,3.0,0.0,1.0,0.0,145.0,140.0,120.0,110.0,150.0,135.0,12.0,30.0,31.0,6.0,17.0,19.0,34,7,0.0,1000,1,143,-28,10150,10142,0.0,30.0,3,3.0,180.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,300.0,10.0,0.0,9999.0,0,0,0.0,0,0.0,0,0.0,310.0,8.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,310.0,8.0,0.0,9999.0,0,2,30.0,4,150.0,0,400.0,310.0,8.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0
20005,4.0,6.0,3.0,2.0,2020,4,13,13,1,1.0,1.0,5.0,3.0,0.0,3.0,145.0,140.0,120.0,110.0,150.0,135.0,12.0,30.0,31.0,6.0,17.0,19.0,30,12,0.0,1000,1,142,11,10147,10139,0.0,41.0,5,5.0,180.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,300.0,10.0,0.0,9999.0,0,0,0.0,0,0.0,0,0.0,310.0,8.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,310.0,8.0,0.0,9999.0,0,2,30.0,4,150.0,0,400.0,310.0,8.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0
20006,8.0,7.0,6.0,6.0,2020,4,13,14,1,1.0,1.0,4.0,3.0,2.0,0.0,145.0,140.0,120.0,110.0,150.0,135.0,12.0,30.0,31.0,6.0,17.0,19.0,28,11,0.0,1000,1,157,5,10142,10134,0.0,36.0,5,5.0,180.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,280.0,15.0,25.0,9999.0,0,0,0.0,0,0.0,0,0.0,300.0,15.0,25.0,9999.0,0,0,400.0,0,400.0,0,400.0,310.0,8.0,0.0,9999.0,0,2,30.0,4,150.0,0,400.0,310.0,8.0,0.0,9999.0,0,4,30.0,0,400.0,0,400.0
20007,5.0,4.0,4.0,3.0,2020,4,13,15,1,1.0,1.0,8.0,6.0,0.0,0.0,145.0,140.0,120.0,110.0,150.0,135.0,12.0,30.0,31.0,6.0,17.0,19.0,28,13,0.0,1000,1,151,14,10139,10131,0.0,40.0,5,5.0,180.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,280.0,15.0,25.0,9999.0,0,0,0.0,0,0.0,0,0.0,300.0,15.0,25.0,9999.0,0,0,400.0,0,400.0,0,400.0,320.0,15.0,25.0,9999.0,0,2,30.0,4,150.0,0,400.0,320.0,15.0,25.0,9999.0,0,4,30.0,

In [42]:
Data.to_csv('..\\data\\data_raw.csv')

In [41]:
Data

,AAR,EAD,ADR,EDD,year,month,day,hour,DayName,Arpt_cond,P_Airp,P_AAR,P_ADR,Arrival_remainder,Departure_remainder,WD_400,WD_500,WD_700,WD_850,WD_925,WD_1000,WS_400,WS_500,WS_700,WS_850,WS_925,WS_1000,WD,WSPD,WS_GST,VIS,WC,TMP,TD,PS,PA,RN,HM,CA_TOT,CLA_1LYR,BASE_1LYR,CLA_2LYR,BASE_2LYR,CLA_3LYR,BASE_3LYR,CLA_4LYR,BASE_4LYR,RVR,WDIR_t6,WSPD_t6,WG_t6,VIS_t6,WC_t6,CLA_1LYR_t6,BASE_1LYR_t6,CLA_2LYR_t6,BASE_2LYR_t6,CLA_3LYR_t6,BASE_3LYR_t6,WDIR_t12,WSPD_t12,WG_t12,VIS_t12,WC_t12,CLA_1LYR_t12,BASE_1LYR_t12,CLA_2LYR_t12,BASE_2LYR_t12,CLA_3LYR_t12,BASE_3LYR_t12,WDIR_t18,WSPD_t18,WG_t18,VIS_t18,WC_t18,CLA_1LYR_t18,BASE_1LYR_t18,CLA_2LYR_t18,BASE_2LYR_t18,CLA_3LYR_t18,BASE_3LYR_t18,WDIR_t24,WSPD_t24,WG_t24,VIS_t24,WC_t24,CLA_1LYR_t24,BASE_1LYR_t24,CLA_2LYR_t24,BASE_2LYR_t24,CLA_3LYR_t24,BASE_3LYR_t24
0,3.0,2.0,11.0,11.0,2018,1,1,0,1,1.0,1.0,0.0,0.0,0.0,0.0,285.0,295.0,300.0,320.0,325.0,245.0,72.0,47.0,43.0,28.0,19.0,3.0,34,5,0.0,1000,1,-26,-115,10268,10259,0.0,50.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
1,0.0,1.0,4.0,5.0,2018,1,1,1,1,1.0,1.0,3.0,11.0,0.0,0.0,285.0,295.0,300.0,320.0,325.0,245.0,72.0,47.0,43.0,28.0,19.0,3.0,3,1,0.0,1000,1,-21,-117,10266,10257,0.0,47.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
2,0.0,1.0,4.0,4.0,2018,1,1,2,1,1.0,1.0,0.0,4.0,1.0,1.0,285.0,295.0,300.0,320.0,325.0,245.0,72.0,47.0,43.0,28.0,19.0,3.0,35,1,0.0,1000,1,-36,-113,10266,10257,0.0,55.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
3,2.0,0.0,1.0,1.0,2018,1,1,3,1,1.0,1.0,0.0,4.0,1.0,0.0,285.0,295.0,300.0,320.0,325.0,245.0,72.0,47.0,43.0,28.0,19.0,3.0,0,0,0.0,1000,1,-42,-111,10269,10260,0.0,58.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
4,18.0,16.0,0.0,1.0,2018,1,1,4,1,1.0,1.0,2.0,1.0,0.0,0.0,285.0,295.0,300.0,320.0,325.0,245.0,72.0,47.0,43.0,28.0,19.0,3.0,19,1,0.0,1000,1,-41,-104,10271,10262,0.0,61.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,11.0,9.0,11.0,8.0,2020,12,31,19,4,1.0,1.0,5.0,13.0,9.0,0.0,340.0,325.0,355.0,270.0,280.0,325.0,13.0,9.0,5.0,39.0,29.0,16.0,32,6,0.0,1000,1,-53,-119,10258,10249,0.0,59.0,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,350.0,6.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,330.0,15.0,25.0,9999.0,0,4,30.0,0,400.0,0,400.0,330.0,15.0,25.0,9999.0,0,4,30.0,0,400.0,0,400.0
26300,7.0,6.0,5.0,6.0,2020,12,31,20,4,1.0,1.0,11.0,11.0,0.0,0.0,340.0,325.0,355.0,270.0,280.0,325.0,13.0,9.0,5.0,39.0,29.0,16.0,31,11,0.0,1000,1,-48,-110,10263,10254,0.0,61.0,1,1.0,30.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,350.0,6.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,330.0,15.0,25.0,9999.0,0,4,30.0,0,400.0,0,400.0,330.0,15.0,25.0,9999.0,0,4,30.0,0,400.0,0,400.0
26301,3.0,3.0,9.0,8.0,2020,1